In [1]:
import sys
print(sys.executable)
from azure.cosmos import CosmosClient, PartitionKey, exceptions
from tqdm import tqdm
import json 

/usr/local/bin/python3


In [2]:
ACCOUNT_URI = "https://lolpredict.documents.azure.com:443/"
ACCOUNT_KEY = "UcLcT6bvFUsk677XfibNhJIIqiAstRkaWWrqTJaQOCmVqx0IN0J1ufYdpbsarowutD6nMSKaaLS1NqcD0QDIdQ=="
client = CosmosClient(ACCOUNT_URI, credential=ACCOUNT_KEY)


## Create League Database

In [21]:
def create_database(database_name):
    try:
        database = client.create_database(database_name)
    except exceptions.CosmosResourceExistsError:
        database = client.get_database_client(database_name)
    return database

def create_container(database, container_name):
    try:
        container = database.create_container(id=container_name,    partition_key=PartitionKey(path="/league"))
    except exceptions.CosmosResourceExistsError:
        container = database.get_container_client(container_name)
    except exceptions.CosmosHttpResponseError:
        raise
    return container

def get_database(database_name):
    return client.get_database_client(database_name)

def get_container(database, container_name):
    return database.get_container_client(container_name)

def insert_item(container,data):
    container.upsert_item(data)

In [4]:
database = create_database("league_all_data")
champion_container = create_container(database, "champions")


## send champion data to azure 

In [ ]:
count = 0
with open("champion.json","rb") as champ:
    champ_json = json.load(champ)
    #load through each champ

In [ ]:
for c in tqdm(champ_json['data'].keys()):
    c_data_raw = champ_json['data'][c]
    champ_dict = {}
    champ_dict['id'] = c_data_raw['key']
    champ_dict['name'] = c
    champ_dict['stats'] = c_data_raw['stats']
    champ_dict['tags'] = c_data_raw['tags']
    insert_item(champion_container,champ_dict)
print("Done insert data to cosmosDB") 

## Querying champion data from CosmosDB

In [5]:
query = "SELECT * FROM mycontainer r WHERE r.id='23'"
for item in champion_container.query_items(query=query, enable_cross_partition_query=True):
    print(json.dumps(item, indent=True))
    print("-"*10)

{
 "id": "23",
 "name": "Tryndamere",
 "stats": {
  "hp": 625.64,
  "hpperlevel": 98,
  "mp": 100,
  "mpperlevel": 0,
  "movespeed": 345,
  "armor": 33,
  "armorperlevel": 3.1,
  "spellblock": 32.1,
  "spellblockperlevel": 1.25,
  "attackrange": 125,
  "hpregen": 8.5,
  "hpregenperlevel": 0.9,
  "mpregen": 0,
  "mpregenperlevel": 0,
  "crit": 0,
  "critperlevel": 0,
  "attackdamage": 69,
  "attackdamageperlevel": 3.7,
  "attackspeedperlevel": 2.9,
  "attackspeed": 0.67
 },
 "tags": [
  "Fighter",
  "Assassin"
 ],
 "_rid": "nXoLAJNWKLB5AAAAAAAAAA==",
 "_self": "dbs/nXoLAA==/colls/nXoLAJNWKLA=/docs/nXoLAJNWKLB5AAAAAAAAAA==/",
 "_etag": "\"0000d104-0000-0400-0000-5dcf4ffe0000\"",
 "_attachments": "attachments/",
 "_ts": 1573867518
}
----------


## Creating player and match container 

In [6]:
import league_api
import importlib
importlib.reload(league_api)

<module 'league_api' from '/Users/wazzie27/Documents/LOL_CSP/LOL/league_predict/league_api.py'>

In [7]:
# create container 
player_container = create_container(database,"players")
match_container = create_container(database,"matches")

In [8]:
#got list of players of diamond I
league = league_api.LeagueAPI()
league.set_key("RGAPI-82ec18c1-a18d-469d-b54e-04b5da131794")
player_list = league.get_player_list("DIAMOND","I")

Got 205 players


In [19]:
def get_games_1_player(summonerId,game_num):
    player_dict = {}
    player_dict['id'] = str(summonerId)
    account_id = league.get_account_id(summonerId)
    player_mastery = league.get_champion_mastery(summonerId)
    player_dict['mastery'] = player_mastery
    match_hist = league.get_player_match_hist(account_id)
    all_other_players = []
    final_matches = []
    for match in match_hist[:game_num]:
        game_stats = league.get_game_stats(match)
        try:
            match_dict = {}
            match_dict["id"] = str(game_stats['gameId'])
            match_dict["stats"] = game_stats
            players_in_game = league.get_players_id_1_match(game_stats,exclude=summonerId) 
            #exclude the player that call the query
            all_other_players += players_in_game
            insert_item(match_container,match_dict)
            final_matches.append(match_dict['id'])
        except:
            print("cant get match hist for this")
            pass
    all_other_players = list(set(all_other_players)) #list of players play with the current summonerId
    player_dict['matches'] = final_matches
    insert_item(player_container,player_dict) #send to cosmos
    return all_other_players,len(final_matches)

## main program to send to 2 containers

In [22]:
#we only query data for 1 user first
#for each player, we query his most 10 recent matches
#for each match, we query match history of 10 players 
#for each of those player, we query his most 5 recent matches
#and then we stop there => around 60 matches query = 1 match
# we sacrifice data quality for quantity, getting as much matches as possible
match_count = 0
for player in tqdm(player_list):
    print("seed user {}".format(player))
    all_other_players,len_matches = get_games_1_player(player,10)
    placeholder = None
    print("query {} other players".format(len(all_other_players)))
    for o_pl in all_other_players: #for each other player, query 5 matches
        placeholder = get_games_1_player(o_pl,5)
    match_count += len_matches
    print("Done for user {}, now we have {} full matches".format(player,match_count))






  0%|          | 0/205 [00:00<?, ?it/s]

seed user EuXHkripW2Qav3H2l1uGWapRCMgwQh4fHIVFybcTCuTk8XX_
3207690957
3207597356


KeyboardInterrupt: 